In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 70)
pd.set_option('display.max_rows',150)

In [3]:
df = pd.read_csv('input/cierres_nc/210730-1136-cierres_nc.csv', sep=';', dtype='object')

In [4]:
f4 = pd.read_csv('input/cierres_nc/210730-1136-f4.csv', sep=';', dtype='object')

In [5]:
df.shape

(60219, 14)

In [6]:
cerrado = df[df['esmc']=='cerrado']

In [7]:
cerrado.shape

(57433, 14)

In [8]:
eac = cerrado[cerrado['tipmc']=='se asocia f4 dado de baja por producto entregado a cliente'].reset_index(drop=True)

In [9]:
eac.shape

(1561, 14)

Unir f4 y cnc

In [10]:
ef = eac.merge(f4, how='left', left_on=['f4', 'upc'], right_on=['nro_red_inventario', 'upc'])

In [11]:
ef.loc[ef.nro_red_inventario.isna()].shape

(725, 21)

Tiene # de F4?

In [12]:
na = ef.loc[ef.f4.isna()]
ef.loc[na.index,'D2'] = 'No tiene número de F4'
notna = ef.loc[ef.f4.notna()]

F4 único 

In [13]:
ef.loc[(ef.D2.isna())& (ef.duplicated(['f4'], keep=False))].shape

(220, 22)

In [14]:
ef.loc[(ef.D2.isna())& (ef.duplicated(['f4'], keep=False)), 'D2'] = 'F4 tiene varias NC'

Fecha de creación diferente a julio

In [15]:
ef.fecha_creacion.unique()

array([nan, '12-jul-2021', '13-jul-2021', '11-jun-2021', '14-jul-2021',
       '05-may-2021'], dtype=object)

In [16]:
ef.loc[(ef.D2.isna())& (ef.fecha_creacion=='05-may-2021')].shape 

(1, 22)

In [17]:
ef.loc[(ef.D2.isna())& (ef.fecha_creacion=='05-may-2021') ,'D2'] = 'F4 con fecha de creación diferente a 07/21'

In [18]:
ef.loc[(ef.D2.isna())& (ef.fecha_creacion=='11-jun-2021')].shape 

(1, 22)

In [19]:
ef.loc[(ef.D2.isna())&(ef.fecha_creacion=='11-jun-2021'), 'D2'] = 'F4 con fecha de creación diferente a 07/21'

F4 anulados

In [20]:
ef.loc[ef.D2.isna()]['estado'].unique()

array(['registrado', 'anulado'], dtype=object)

In [21]:
ef.loc[(ef.D2.isna())&(ef.estado=='anulado')].shape

(1, 22)

In [22]:
ef.loc[(ef.D2.isna())&(ef.estado=='anulado'), 'D2'] = 'F4 con estado anulado'

Existen F4 en estado registrado? 

In [23]:
ef.loc[(ef.D2.isna())&(ef.estado=='registrado')].shape

(613, 22)

In [24]:
ncc = ef.loc[(ef.D2.isna())&(ef.cantidad_trx_actual!=ef.cantidad)]

In [25]:
ncc.to_excel(f'output/210730-d2-f4-ncc.xlsx', sheet_name=f'nc-f4-base', index=False) 

In [26]:
ef.head()

,cod_aut_nc,local_trx,terminal,local_ant,upc,cantidad_trx_actual,ct,tipo_nc,f5,f11_x,f3,f4,esmc,tipmc,nro_red_inventario,estado,fecha_creacion,destino,linea,cantidad,f11_y,D2
0,1060679519,3000,17,2000,7707001583749,1,41600,PARCIAL,NaN,NaN,NaN,NaN,cerrado,se asocia f4 dado de baja por producto entrega...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No tiene número de F4
1,1060602364,3000,3001,2000,17817789592,1,1812184976,TOTAL,NaN,NaN,NaN,NaN,cerrado,se asocia f4 dado de baja por producto entrega...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No tiene número de F4
2,1060488739,3000,3001,2000,43917005508,1,90743033,TOTAL,NaN,NaN,NaN,NaN,cerrado,se asocia f4 dado de baja por producto entrega...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No tiene número de F4
3,1060680379,2000,23,2000,8002713130472,1,19609,PARCIAL,NaN,NaN,NaN,NaN,cerrado,se asocia f4 dado de baja por producto entrega...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No tiene número de F4
4,1060734966,3000,17,2000,50875827844,1,198085078,PARCIAL,NaN,NaN,NaN,NaN,cerrado,se asocia f4 dado de baja por producto entrega...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No tiene número de F4


In [27]:
ef.loc[ef.D2.isna(), 'D2'] = 'OK'

In [28]:
list(ef.loc[(ef.D2.isna()), 'destino'])

[]

In [29]:
ef['ct'] = pd.to_numeric(ef['ct'])

In [30]:
ef.to_excel(f'output/210730-d2-f4.xlsx', sheet_name=f'nc-f4-base', index=False) 